## Install Dependencies

In [113]:
#!pip install pandas torch datasets transformers sentence-transformers requests tqdm nltk

In [114]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from collections import Counter
from tqdm.auto import tqdm
import mmh3
import requests
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

## Connect to Pinecone

The hybrid vector index is currently not available in Pinecone python client. So, we will use the Pinecone REST API to communicate with the new index. The ```HybridPinecone``` class below gives you a similar interface to the python-client to communicate with the new index.

In [115]:
class HybridPinecone:
    # initializes the HybridPinecone object
    def __init__(self, api_key, environment):
        # make environment, headers and project_id available across all the function within the class
        self.environment = environment
        self.headers = {'Api-Key': api_key}
        # get project_id
        res = requests.get(
            f"https://controller.{self.environment}.pinecone.io/actions/whoami",
            headers=self.headers
        )
        self.project_id = res.json()['project_name']
        self.host = None

    # creates an index in pinecone vector database
    def create_index(self, index_name, dimension, metric, pod_type):
        # index specification
        params = {
            'name': index_name,
            'dimension': dimension,
            'metric': metric,
            'pod_type': pod_type
        }
        # sent a post request with the headers and parameters to pinecone database
        res = requests.post(
            f"https://controller.{self.environment}.pinecone.io/databases",
            headers=self.headers,
            json=params
        )
        # return the creation status
        return res
    
    # get the project_id for the index and update self.host variable
    def connect_index(self, index_name):
        # set the self.host variable
        self.host = f"{index_name}-{self.project_id}.svc.{self.environment}.pinecone.io"
        res = self.describe_index_stats()
        # return index related information as json
        return res
    
    def describe_index(self, index_name):
        # send a get request to pinecone database to get index description
        res = requests.get(
            f"https://controller.{self.environment}.pinecone.io/databases/{index_name}",
            headers=self.headers
        )
        return res.json()

    # returns description of the index
    def describe_index_stats(self):
        # send a get request to pinecone database to get index description
        res = requests.get(
            f"https://{self.host}/describe_index_stats",
            headers=self.headers
        )
        # return the index description as json
        return res.json()

    # uploads the documents to pinecone database
    def upsert(self, vectors):
        # send a post request with vectors to pinecone database
        res = requests.post(
            f"https://{self.host}/hybrid/vectors/upsert",
            headers=self.headers,
            json={'vectors': vectors}
        )
        # return the http response status
        return res

    # searches pinecone database with the query
    def query(self, query):
        # sends a post request to hybrib vector index with the query dict
        res = requests.post(
            f"https://{self.host}/hybrid/query",
            headers=self.headers,
            json=query
        )
        # returns the result as json
        return res.json()

    # deletes an index in pinecone database
    def delete_index(self, index_name):
        # sends a delete request
        res = requests.delete(
            f"https://controller.{self.environment}.pinecone.io/databases/{index_name}",
            headers=self.headers
        )
        # returns the http response status
        return res

## Create index

In [116]:
# initialize an instance of HybridPinecone class
pinecone = HybridPinecone(
    api_key = "1f136ea0-a50c-4af1-a9ad-93de37970fab",  # app.pinecone.io
    environment = "us-west1-gcp"
)

# choose a name for your index
index_name = "hybrid-search-demo"

In [117]:
# create the index
pinecone.create_index(
    index_name = index_name,
    dimension = 384,
    metric = "dotproduct",
    pod_type = "s1h"
)

<Response [201]>

Now we have created the hybrid vector index using the `"s1h"` hybrid `pod_type`. To connect to the index we must `wait until it is ready`, we can check it's status like so:

In [124]:
pinecone.describe_index(index_name)

{'database': {'name': 'hybrid-search-demo',
  'index_type': 'approximated',
  'metric': 'dotproduct',
  'dimension': 384,
  'replicas': 1,
  'shards': 1,
  'pods': 1,
  'pod_type': 's1h',
  'index_config': {'approximated': {'k_bits': 512}}},
 'status': {'waiting': [],
  'crashed': [],
  'host': 'hybrid-search-demo-94a860f.svc.us-west1-gcp.pinecone.io',
  'port': 433,
  'state': 'Ready',
  'ready': True}}

If the `state` is `'Ready'` we can continue and connect to the index like so:

In [125]:
pinecone.connect_index(index_name)

{'namespaces': {}, 'dimension': 384, 'indexFullness': 0, 'totalVectorCount': 0}

## Load Dataset

We will use a json file containing a web crawl of pinecone.io

In [126]:
df = pd.read_json("Pinecone_io_Webcrawl.json")

ids = [] # id
titles = [] # title
contents = [] # content
urls = [] # url
boosts = [] # boost
blob_texts = [] # derived from titles, bodies, and urls

# Process fields to be used as metadata later
for doc in df.response.docs:

    title = str(doc.get('title'))
    titles.append(title)

    content = str(doc.get('content'))
    contents.append(content[:500])

    url = str(doc.get('url'))
    urls.append(url)

    boost = str(doc.get('boost'))
    boosts.append(boost)

    # Blob text will be used to generate the sparse and dense vectors
    blob_text =  str(doc.get('title')) + ' ' + str(doc.get('content')) + ' ' + str(doc.get('url'))
    blob_texts.append(blob_text)


## Sparse and Dense Vector Creation

In [127]:
# load a sentence transformer model from huggingface
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# create a tokenizer
class Tokenizer:
  def __init__(self):
    self.stemmer = SnowballStemmer('english')

  def encode(self, text):
    words = [self.stemmer.stem(word) for word in word_tokenize(text)]
    ids = [mmh3.hash(word, signed=False) for word in words]
    return dict(Counter(ids))

tokenizer = Tokenizer()

In [128]:
sparse_vectors = []
dense_vectors = []

for blob in blob_texts:
    sparse_vectors.append(tokenizer.encode(str(blob)))
    dense_vectors.append(model.encode([blob], normalize_embeddings=True).tolist())

In [129]:
# Build metadata
meta = []
for url, title, content, boost in zip(urls,titles,contents,boosts):
    metavalue = {'url':url,'title':title,'content':content,'boost':boost}
    meta.append(metavalue)

print(meta[1])

{'url': 'https://www.pinecone.io/learn/', 'title': 'Learning Center | Pinecone', 'content': 'Learning Center | Pinecone\nPricing\nDocs\nLearn\nLearning Center\nCommunity\nCompany\nAbout\nCareers\nPartners\nTrust & Security\nContact\nLog In\nSign Up Free\nPricing\nDocs\nLearn\nCommunity\nCompany\nCareers\nPartners\nTrust & Security\nContact\nLog In\nCreate Account\nToggle menu\nLearn to Love Working with Vector Embeddings\nUnlock the power of machine learning. Our guides will help you conquer vector embeddings and build better applications.\nOur Series\nNatural Language Processing for Semantic Search\nLearn how to bu', 'boost': '2.0329747'}


## Upsert Documents

Now we can go ahead and generate sparse and dense vectors for the full dataset and upsert them along with the metadata to the new hybrid index. We can do that easily as follows:

In [130]:
batch_size = 32

for i in tqdm(range(0, len(blob_texts), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(blob_texts))
    # create unique IDs
    ids = [str(x) for x in range(i, i_end)]

    vectors = []

    for id, sparse, dense, metadata in zip(ids[i:i_end], sparse_vectors[i:i_end], dense_vectors[i:i_end], meta[i:i_end]):
        vectors.append({
            'id': id,
            'sparse_values': sparse,
            'values': dense,
            'metadata': metadata
        })

    # upload the documents to the new hybrid index
    pinecone.upsert(vectors)

# show index description after uploading the documents
pinecone.describe_index_stats()

  0%|          | 0/6 [00:00<?, ?it/s]

{'namespaces': {'': {'vectorCount': 32}},
 'dimension': 384,
 'indexFullness': 0,
 'totalVectorCount': 32}

## Querying

Now we can query the index, providing the sparse and dense vectors of a question, along with a weight for keyword relevance (“alpha”). `Alpha=1` will provide a purely semantic-based search result and `alpha=0` will provide a purely keyword-based result equivalent to BM25. The default value is `0.5`.

Let's write a helper function to execute queries and after that run some queries.

In [131]:
def hybrid_query(question, top_k, alpha):
    # convert the question into a sparse vector
    sparse_vec = generate_sparse_vectors([question])
    # convert the question into a dense vector
    dense_vec = model.encode([question]).tolist()
    # set the query parameters to send to pinecone
    query = {
      "topK": top_k,
      "vector": dense_vec,
      "sparseVector": sparse_vec[0],
      "alpha": alpha,
      "includeMetadata": True
    }
    # query pinecone with the query parameters
    result = pinecone.query(query)
    # return search results as json
    return result

In [132]:
question = "are you in us-east-2"

First, we will do a pure semantic search by setting the alpha value as 1.

In [133]:
hybrid_query(question, top_k=1, alpha=1)

NameError: name 'generate_sparse_vectors' is not defined

The most relevant result from above is the second document with id 711. Now let's try with an alpha value of 0.3.

In [ ]:
hybrid_query(question, top_k=1, alpha=0.3)

The most relevant document is now ranked the highest.

# Delete the Index

In [ ]:
#pinecone.delete_index("hybrid-search-demo")